In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
import csv
import os
from leaf_area_basic_cycles import get_total_leaf_area
from sum_individual_vein_lengths import get_total_vein_length

## Utility Functions 

##### graph_from_data 
allows quickgraph generation if the paths to the node list and edge list are given

In [5]:
def graph_from_data(node_path, edge_path):
    # Extract network id from path
    network_id = os.path.basename(node_path)
    network_id = network_id.split('binary', 1)[0]
    network_id = network_id.split('graph', 1)[0]
    network_id = network_id[:-1]

    # Read node positions from the node file
    nodes = []
    with open(node_path) as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC)
        for row in reader:
            nodes.append({'node_id': int(row[0]), 'x': row[1], 'y': row[2]})

    # Read edge positions from the edge file
    edges = []
    with open(edge_path) as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quoting=csv.QUOTE_NONNUMERIC)
        for row in reader:
            edges.append({'node_id1': int(row[0]), 'node_id2': int(row[1]), 'radius': row[2], 'length': row[3]})

    # Create the graph with the networkx library
    G = nx.Graph()
    for node in nodes:
        G.add_node(node['node_id'], pos=(node['x'], node['y']))
    for edge in edges:
        G.add_edge(edge['node_id1'], edge['node_id2'], weight=edge['radius'], length=edge['length'])
        
    return network_id, G

##### graph_generator
basically just applies graph_from_data to all file paths and returns the graphs one by one. Notice the word yield at the end instead of the usual return of python functions. This tells python that this function is a generator. We can iterate over this generator as if it were a list. Everytime we normally were to access the next list element, the generator yields the next result. This way, there is never more than one graph in the memory. If we created a list of all graphs, this list would probably be several GB in size.
 


In [6]:
def graph_generator():
    data_list = []
    for subset in ['BronxA', 'BronxB']:
        # Get a list of all files in the BronxA or BronxB directory
        file_list = os.listdir('data/networks-{}'.format(subset))
        file_list.sort()  #file_list comes with unsorted files 
        
        # The file names result in the node list always following the edge list,
        # which means that all even numbers correspond to edge lists and all odd
        # numbers to node lists.
        for i in range(len(file_list)//2):
            edge_path = os.path.join('data/networks-{}'.format(subset), file_list[2*i])
            node_path = os.path.join('data/networks-{}'.format(subset), file_list[2*i + 1])
            # Everytime the next element of the generator is accessed, the current step in
            # the loop is executed. After the 'yield' line is finished, the generator will pause
            # until the next element is accessed again. So everytime we access the next element,
            # we raise i to i+1, get the paths for the corresponding files and generate the graph
            # for these files.
              
            yield graph_from_data(node_path, edge_path)   ##Ignores everything after first yield
            print ('first yield')     
            #print (edge_path)
            #yield total_vein_length(edge_path)
            #print ('second yield')
            ###NOTE: Have to restart kernel if any change in generator is done

## Data Analysis


#### Get species name

In [ ]:
species = []    ##CHECK: Make it a function
index = "066" 
with open('data/LABELS_1.txt') as csvfile:   #BronxB LABEL1, BronxA LABEL2
    reader = csv.reader(csvfile, delimiter=' ')  #Works with elements as strings 
    for row in reader:
        if row[1] == index: 
            species.append(' '.join(row))

#### Leaf Graph

In [7]:
generator = graph_generator()

In [8]:
_, G = next(generator)
pos = nx.get_node_attributes(G,'pos')  ##NOTE: Check nx.get_ (attributes)
#nx.draw(G, pos=pos, node_size=0.1)

## Features

### 1 Geometrical

### 1.1 Number of nodes and edges


In [ ]:
#Calculate the number of nodes and edges for each network and save them to files
with open('features/n_nodes.txt', 'w') as node_file, open('features/n_edges.txt', 'w') as edge_file:
    for network_id, graph in graph_generator():     #Check this access to graph_generator
        node_file.write(network_id + '  ' + str(nx.number_of_nodes(graph)) + '\n')  #nx.number_of_nodes() function
        edge_file.write(network_id + '  ' + str(nx.number_of_edges(graph)) + '\n')  #Two spaces 
        
###NOTE: Have to pull features from github before running script
##IDEA: Can calculate every feature on the same loop 

### 1.2 Average Node Degree

In [ ]:
# Calculate the average node degree, i.e. the average number of edges that are connected to a node,
# for each network and write them to a file
with open('features/average_node_degree.txt', 'w') as node_file:  
    for network_id, graph in graph_generator():     #Check this access to graph_generator
        average_node_degree = np.mean(list(graph.degree().values()))
        node_file.write(network_id + '  ' + str(average_node_degree) + '\n')

###  1.3 Vein density


Vein density **σ** is calculated by summing the total length of all veins and dividing by total leaf area.

In [10]:
total_vein_length = get_total_vein_length(G)
total_leaf_area = get_total_leaf_area(G)

vein_density = total_vein_length/total_leaf_area

#print('vein density',vein_density)
#print('total vein length',total_vein_length)
#print('leaf area',total_leaf_area)

vein density 0.00025849100767
total vein length 2328902.8143422347
leaf area 9009608633.33


### Topological